In [23]:
import spacy
import pandas as pd

# Load laptop data from CSV
laptops_df = pd.read_csv("laptops_cleaned.csv")
nlp = spacy.load("en_core_web_lg")

def preprocess_text(text):
    """Preprocess the text using spaCy."""
    doc = nlp(text)
    # Extract price, RAM, and GPU information
    price = None
    ram = None
    gpu = None
    # Initialize variables to store the words between "and" and "GPU"
    words_between = []
    # Iterate through tokens in the document
    for i, token in enumerate(doc):
        # Check if the token text is 'price'
        if token.text.lower() == '$':
            # Check if the next token resembles a number
            if i + 1 < len(doc) and doc[i + 1].like_num:
                # Extract the numerical value of the next token
                price = float(doc[i + 1].text)
        elif token.text.lower() == 'gb':
            # Check if the next token resembles a number
            if i + 1 < len(doc) and doc[i - 1].like_num:
                # Extract the numerical value of the next token
                ram = int(doc[i - 1].text)

        # Check if the token text is 'nvidia' or 'amd'
        if token.text.lower() in ['nvidia', 'amd', 'intel']:
            words_between.append(token.text)
            for j in range(i + 1, len(doc)):
                if doc[j].text.lower() in [".", "and", "gpu", ","]:
                    break
                else:
                    words_between.append(doc[j].text)

    # Convert the list of words between 'and' and 'GPU' to a string
    gpu = ' '.join(words_between)
    return price, ram, gpu

def recommend_laptop(price, ram, gpu):
    """Recommend a laptop based on price, RAM, and GPU."""
    # Filter laptops based on user input
    filtered_laptops = laptops_df
    if price:
        filtered_laptops = filtered_laptops[filtered_laptops['Price_euros'] <= price]
    if ram:
        filtered_laptops = filtered_laptops[filtered_laptops['Ram'] >= ram]
    if gpu:
        filtered_laptops = filtered_laptops[filtered_laptops['Gpu'] == gpu]
    print(len(filtered_laptops))
    # Sort laptops by price and return the top recommendation
    if not filtered_laptops.empty:
        return filtered_laptops.sort_values(by='Price_euros').iloc[0]
        #return filtered_laptops
    else:
        return "No laptops match the specified criteria."

sample_text = "I want a laptop with a ram of 8GB, around $1000 and Nvidia GeForce 930MX gpu"
sample_price, sample_ram, sample_gpu = preprocess_text(sample_text)

# Recommend a laptop based on sample inputs
recommendation = recommend_laptop(sample_price, sample_ram, sample_gpu)
print(recommendation)

9
Unnamed: 0.1                                964
Unnamed: 0                                  964
laptop_ID                                   978
Company                                      HP
Product                             ProBook 450
TypeName                               Notebook
Inches                                     15.6
ScreenResolution                       1366x768
Cpu                 Intel Core i5 7200U 2.50GHz
Ram                                           8
Memory                                  1TB HDD
Gpu                        Nvidia GeForce 930MX
OpSys                                Windows 10
Weight                                   2.04kg
Price_euros                               846.5
Name: 964, dtype: object
